In [1]:
import xarray as xr
from pathlib import Path
import glob

## Create Synthetic Data from Xarray

On datarmor compute node, i can't access internet/(can not access to synthetic data example) thus used real data available on datalake.
/home/ref-oc-public/modeles_marc/f1_e4000/best_estimate/2014/MARC_F1-MARS3D-MANGAE4000_2014123120*Z.nc
These are hourly netcdf computed output.

Let's create a tarbar for the created netCDF files

In [9]:
!mkdir -p data/realdata

In [10]:
cp  /home/ref-oc-public/modeles_marc/f1_e4000/best_estimate/2014/MARC_F1-MARS3D-MANGAE4000_20141231T2*Z.nc data/realdata/.

In [11]:
!tar Pcvf data/realdata.tar data/realdata/*

data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc
data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc
data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc
data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc


## ratarmount

We will use [ratarmount](https://github.com/mxmlnkn/ratarmount) to create an index file with file names, ownership, permission flags, and offset information to be stored at the TAR file's location. Once the index is created, ratarmount then offers a FUSE mount integration for easy access to the files.

**NOTE:** Since `ratarmount` uses FUSE to mount the TAR file as a "filesystem in user space", you will need FUSE installed.  On OSX, you will need to install [osxfuse](https://osxfuse.github.io/) *by hand*.  On Linux, you can install `libfuse` using `conda`, if it is not already installed on your system.

**NOTE:** If you have `libfuse` on your system and it is *older* than October 19, 2018 (i.e., < 3.3.0 for `fuse3` or < 2.9.9 for `fuse2`), and you have either Lustre or GPFS filesystems, `ratarmount` will fail with an error saying that your filesystem is unsupported.  The solution is to upgrade to a newer version of `libfuse`.

**NOTE:** If you install the `libfuse` Conda-Forge package on a Linux system, then you need to set the `FUSE_LIBRARY_PATH` environment variable to the location of the `libfuse.so` library file (e.g., `export FUSE_LIBRARY_PATH=/path/to/libfuse3.so`).  If you do not do this, then `fusepy` (another dependency of `ratarmount`) will use the system `libfuse.so` file, which might be old.

**NOTE:** Currently, the Conda-Forge version of `libfuse` does *not* build the `libfuse` utilities such as `fusermount3`.  However, `fusepy` uses these utility functions under the hood when trying to mount the userspace filesystem.  If you install the most recent version of `libfuse` and properly set the location of `libfuse` so that `fusepy` can find it (i.e., `FUSE_LIBRARY_PATH`), you will get an error the `fusermount3` cannot be found.

In [12]:
%%time
!/home1/datahome/todaka/conda-env/xtar-dev/bin/ratarmount --recreate-index data/realdata.tar mounted_dataset

Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data/realdata.tar ...
Creating new SQLite index database at /home1/datahome/todaka/git/xtar/notebooks/data/realdata.tar.index.sqlite
Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data/realdata.tar took 0.00s
Writing out TAR index to /home1/datahome/todaka/git/xtar/notebooks/data/realdata.tar.index.sqlite took 0s and is sized 24576 B
CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 339 ms


In [17]:
#prepare test GPFS
!mkdir -p ${DATAWORK}/xtar
!ln -sf ${DATAWORK}/xtar data_gpfs
!cp -R data/realdata* data_gpfs/.

In [20]:
#prepare test Lustre
!mkdir -p ${SCRATCH}/xtar
!ln -sf ${SCRATCH}/xtar data_lustre
!cp -R data/realdata* data_lustre/.


In [21]:
#test GPFS
!/home1/datahome/todaka/conda-env/xtar-dev/bin/ratarmount --recreate-index data_gpfs/realdata.tar data_gpfs/mounted_dataset

Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data_gpfs/realdata.tar ...
Creating new SQLite index database at /home1/datahome/todaka/git/xtar/notebooks/data_gpfs/realdata.tar.index.sqlite
Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data_gpfs/realdata.tar took 0.13s
Writing out TAR index to /home1/datahome/todaka/git/xtar/notebooks/data_gpfs/realdata.tar.index.sqlite took 0s and is sized 24576 B


In [23]:
#test Lustre
#test GPFS
!/home1/datahome/todaka/conda-env/xtar-dev/bin/ratarmount --recreate-index data_lustre/realdata.tar data_lustre/mounted_dataset

Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data_lustre/realdata.tar ...
Creating new SQLite index database at /home1/datahome/todaka/git/xtar/notebooks/data_lustre/realdata.tar.index.sqlite
Creating offset dictionary for /home1/datahome/todaka/git/xtar/notebooks/data_lustre/realdata.tar took 0.01s
Writing out TAR index to /home1/datahome/todaka/git/xtar/notebooks/data_lustre/realdata.tar.index.sqlite took 0s and is sized 24576 B


For comparison, this is how long it takes to `list` the original data directory.

In [14]:
%%time
mounted_dir = Path("mounted_dataset/data/realdata/")
list(mounted_dir.iterdir())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.07 ms


[PosixPath('mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc'),
 PosixPath('mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc'),
 PosixPath('mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc'),
 PosixPath('mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc')]

**NFS=> tar 1.07 ms  real file 1.18ms

In [24]:
%%time
mounted_dir = Path("data_gpfs/mounted_dataset/data/realdata/")
list(mounted_dir.iterdir())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.31 ms


[PosixPath('data_gpfs/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc'),
 PosixPath('data_gpfs/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc'),
 PosixPath('data_gpfs/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc'),
 PosixPath('data_gpfs/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc')]

In [27]:
%%time
mounted_dir = Path("data_gpfs/realdata/")
list(mounted_dir.iterdir())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 853 µs


[PosixPath('data_gpfs/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc'),
 PosixPath('data_gpfs/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc'),
 PosixPath('data_gpfs/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc'),
 PosixPath('data_gpfs/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc')]

**GPFS=> tar 1.31 ms  real file 853 µs

In [25]:
%%time
mounted_dir = Path("data_lustre/mounted_dataset/data/realdata/")
list(mounted_dir.iterdir())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.18 ms


[PosixPath('data_lustre/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc'),
 PosixPath('data_lustre/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc'),
 PosixPath('data_lustre/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc'),
 PosixPath('data_lustre/mounted_dataset/data/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc')]

In [28]:
%%time
mounted_dir = Path("data_lustre/realdata/")
list(mounted_dir.iterdir())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 925 µs


[PosixPath('data_lustre/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2000Z.nc'),
 PosixPath('data_lustre/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2200Z.nc'),
 PosixPath('data_lustre/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2100Z.nc'),
 PosixPath('data_lustre/realdata/MARC_F1-MARS3D-MANGAE4000_20141231T2300Z.nc')]

**GPFS=> tar 1.18 ms  real file 925 µs

**slightly  slower to list the directory contents 

Following test not yet done on datarmor

## Benchmarks

In [ ]:
from dask.distributed import performance_report, Client

In [ ]:
client = Client()
client

### Original netCDF files

In [ ]:
ds_orig = xr.open_mfdataset("data/air/*.nc", combine='nested', concat_dim='member_id')
ds_orig

In [ ]:
ds_orig.air.data.visualize()

### Mounted netCDF files from the tar archive

In [ ]:
ds_mntd = xr.open_mfdataset("mounted_air_dataset/data/air/*.nc", combine='nested', concat_dim='member_id')
ds_mntd

In [ ]:
ds_mntd.air.data.visualize()

### Benchmark: Yearly Averages

In [ ]:
%%time
ds_orig.groupby('time.year').mean(['time', 'member_id']).compute()

In [ ]:
%%time
ds_mntd.groupby('time.year').mean(['time', 'member_id']).compute()

### Dask Performance Reports

In [ ]:
with performance_report(filename="dask-perf-report-original.html"):
    ds_orig.groupby('time.year').mean(['time', 'member_id']).compute()

In [ ]:
with performance_report(filename="dask-perf-report-mounted.html"):
    ds_mntd.groupby('time.year').mean(['time', 'member_id']).compute()

In [ ]:
from IPython.display import HTML

In [ ]:
display(HTML("dask-perf-report-original.html"))

In [ ]:
display(HTML("dask-perf-report-mounted.html"))